In [ ]:
# Copyright 2019 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

![](http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png)

# End-to-End BERT (Training) 
이 챕터는 TensorFlow BERT Training 과정을 안내하기 위해 작성되었습니다. 이 문서의 목차는 다음과 같습니다. 

* [1. 모델 아키텍처](#model)
* [2. 데이터셋](#dataset)
* [3. NVIDIA가 제공하는 Training  기법 특징](#features)
* [4. BERT Training 시작하기](#training)
* [5. Training 성능](#performance)
* [6. 결론](#conclusions)

<a id='model'></a>
## 1. 모델 아키텍처
BERT 의 Training 과정은 크게  Pre-training 단계와 Fine-Tuning 단계로 구분됩니다. Pre-Training단계에서는 방대한 데이터셋으로 부터 언어와 문장 자체에 대한 이해도를 높이고, Fine-Tuning 단계에서는 pre-trained된 파라미터들을 downstream 작업을 통해 특화된 작업을 이해할수 있게 해줍니다.
<img src="../data/images/bert_pipeline.png?raw=true" width="700">
그림 1. BERT Training pipeline

그림 1은 BERT Training pipeline으로 Pre-Training 단계와 Fine-Tuning 단계를 나타냅니다.  Pre-Training 단계에서는 large scale의 unannotated 데이터셋을 이용해 unsupervised 방법으로 Training 을 진행합니다. Fine-Tuning 단계에서는  일반적으로 하나 또는 두개 의 레이어를 추가한 후 특정한 작업을 위해 Training 을 하게 되는데, 특정한 작업을 위한 annotated 데이터를 사용하고 Pre-Trained 모델의 weight를 기반으로 Training 을 진행합니다.


<a id='dataset'></a>
## 2. 데이터셋

전체 데이터셋 용량은 170GB 이상이고, 다운로드 하는데 15시간 이상 소요됩니다. BookCorpus 서버가 일시적으로 과부화되어 HTTP 403 이나 503 오류를 발생시킬수 있습니다. 따라서 누락된 파일은 나중에 다시 다운 받으시거나 무시하시길 바랍니다. 데이터셋은 크게 Pre-training을 위한 데이터셋과 Fine-tuning을 위한 데이터셋으로 나뉠 수가 있습니다. 
* Pre-training을 위한 데이터셋: [Wikipedia](https://dumps.wikimedia.org/) (2.5B words) , [BookCorpus](http://yknzhu.wixsite.com/mbweb) (800M words)
* Fine-Tuing을 위한 데이터셋:  [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) (Stanford Question Answering Dataset), [GLUE](https://gluebenchmark.com/) (The General Language Understanding Evaluation benchmark)


<a id='features'></a>
## 3. NVIDIA가 제공하는 BERT Training 기법 특징
다음에 소개할 기술들은 추가적인 노력없이 BERT Training 시간을 단축할 수 있습니다. 

### a. Multi-GPU Training
Multi-GPU Training을 위해 horovod를 사용하였습니다. Horovod는 NCCL을 사용하여 효율적인 Multi-GPU training 환경을 제공합니다. 자세한 내용은 [TensorFlow 튜토리얼](https://github.com/horovod/horovod/#usage)을 참고해 주시길 바랍니다.


### b. Automatic Mixed Precision (AMP) Training

TensorFlow Automatic Mixed Precision (TF-AMP)를 사용하여 Mixed-precision  Training 을 수행합니다. 환경 변수에 의해 제어되며 추가적인 소스 코드 변경 없이 자동적으로 그래프를 재 작성하고 Loss scaling을 수행하여 Training  시간을 단축시켜줍니다.

![](https://developer.nvidia.com/sites/default/files/dev-ai-tech-amp-workflow-graphic-950353-r7-web.png)

그림 2. Automatic Mixed Precision Training

그림 2는 Automatic Mixed Precision Training 단계를 설명한 것으로, Mixed-precision Training 을 사용하려면 두 단계가 필요합니다. FP32의 데이터 타입을 FP16의 데이터타입으로 변경하고, FP16으로 변경함에 따라 gradient 값 손실이 발생하는데 손실되는 gradient 값을 보존하기 위해 loss scaling을 사용합니다.

Mixed-Precision Training의 장점은 다음과 같습니다. 
* 텐서코어를 사용하여 Linear 또는 convolutional 레이어 연산과 같은 math-intensive 연산의 속도를 향상
* FP16 precision 은 FP32 precision 보다 절반의 메모리를 사용하여 메모리 절약 효과
* Training 모델의 메모리 요구량을 줄여줌으로써 더 큰 모델이나 큰 mini-batch 사용 가능

Mixed-precision Training 에 대한 자세한 내용은 다음 링크를 참고 하시길 바랍니다.
* [Mixed-precision Training 논문](https://docs.nvidia.com/deeplearning/dgx/tensorflow-user-guide/index.html#tfamp)과 [Mixed-precsision Training 문서](https://docs.nvidia.com/deeplearning/dgx/tensorflow-user-guide/index.html#tfamp)
* [Mixed-precision Training 기법 블로그](https://docs.nvidia.com/deeplearning/dgx/tensorflow-user-guide/index.html#tfamp)
* [TensorFlow Automatic Mixed-precision User guide 문서](https://docs.nvidia.com/deeplearning/dgx/tensorflow-user-guide/index.html#tfamp)

### c. LAMB (Layerwise Adaptive Moments based optimizer)
[LAMB](https://arxiv.org/abs/1904.00962)는 Large batch 최적화 기법으로, large mini-batch를 사용하여 딥러닝 모델을 학습 시 가속화하는데 도움을 줍니다. 저희는 gradient pre-normalization과 bias correction 추가하여 LAMB (NVLAMB)를 구현했습니다. 보다 자세한 내용은 [여기](https://medium.com/nvidia-ai/a-guide-to-optimizer-implementation-for-bert-at-scale-8338cc7f45fd)를 참조해주세요.


<a id='training'></a>
## 4. BERT Training 시작하기

### Step 1. Docker build 하기
BERT Training 환경을 위해 docker image를 생성하고 빌드합니다. CUDA 10.1과 TensorFlow 1.14.0 기반의 nvcr.io/nvidia/tensorflow:19.08-py3 이미지를 사용해서 빌드했으며 자세한 환경은 [여기](https://docs.nvidia.com/deeplearning/frameworks/tensorflow-release-notes/rel_19.08.html#rel_19.08)를 참고하시길 바랍니다. 

In [ ]:
import os
os.chdir("./..")

In [ ]:
!pwd

In [ ]:
%%bash
docker build . --rm -t bert

### Step 2. Dataset 다운로드 및 TFRecord 생성하기

In [ ]:
%%bash
docker rm -f bert_training
docker run -d --runtime=nvidia -v $PWD:/workspace/bert \
    --rm --shm-size=1g --ulimit memlock=-1 \
    --ulimit stack=67108864 --ipc=host -t -i \
    --name bert_training \
    bert 
docker exec -t bert bash data/create_datasets_from_start.sh

Pre-training을 위한 bokscorpus와 wikicorpus_en 데이터셋을 합치고 TFRecord 셋으로 만드는 과정과 Fine-tuning을 위한 SQuAD, GRUE 데이터셋을  다운로드 하는 과정을 포함합니다. 많은 시간이 소요되므로 반드시 여유시간을 가지고 실행시켜주세요. 데이터셋 다운로드 Link가 깨질수 있으니 다시 시도하거나, 깨진 링크는 무시하셔도 됩니다.

`data/create_datasets_from_start.sh` 스크립트가 포함하는 코드 구성은 다음과 같습니다. 
TFRecord 셋으로 만드는 과정에서는 두 파라미터 max_seq_length 와 max_predictions_per_seq를 설정해주시면 됩니다. 

데이터 생성 스크립트에서는 다음과 같은 파라미터 값을 사용하여 TFRecord 셋을 생성합니다.
 
| max_seq_length | max_predictions_per_seq |
|----------------|-------------------------|
|   128          |           20            |
|   512          |           80            |


In [ ]:
#data/create_datasets_from_start.sh 구성
# export BERT_PREP_WORKING_DIR="${BERT_PREP_WORKING_DIR}"

# # Download
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action download --dataset bookscorpus
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action download --dataset wikicorpus_en

# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action download --dataset google_pretrained_weights  # Includes vocab

# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action download --dataset squad
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action download --dataset "CoLA"
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action download --dataset "MRPC"
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action download --dataset "MNLI"


# # Properly format the text files
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action text_formatting --dataset bookscorpus
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action text_formatting --dataset wikicorpus_en


# # Shard the text files (group wiki+books then shard)
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action sharding --dataset books_wiki_en_corpus


# # Create TFRecord files Phase 1
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action create_tfrecord_files --dataset books_wiki_en_corpus --max_seq_length 128 \
#  --max_predictions_per_seq 20 --vocab_file ${BERT_PREP_WORKING_DIR}/download/google_pretrained_weights/uncased_L-24_H-1024_A-16/vocab.txt


# # Create TFRecord files Phase 2
# python3 ${BERT_PREP_WORKING_DIR}/bertPrep.py --action create_tfrecord_files --dataset books_wiki_en_corpus --max_seq_length 512 \
#  --max_predictions_per_seq 80 --vocab_file ${BERT_PREP_WORKING_DIR}/download/google_pretrained_weights/uncased_L-24_H-1024_A-16/vocab.txt

개별 실행 코드는 다음과 같습니다. 

In [ ]:
# %%bash
# docker exec -t bert python3 data/bertPrep.py --action download --dataset bookscorpus

데이터셋과 TFRecord 셋은 기본적으로 다음과 같은 위치에 다운로드 되고 생성됩니다.
* SQuAD v1.1 - `data/download/squad/v1.1`
* SQuAD v2.0 - `data/download/squad/v2.0`
* GLUE The Corpus of Linguistic Acceptability (CoLA) - `data/download/CoLA`
* GLUE Microsoft Research Paraphrase Corpus (MRPC) - `data/download/MRPC`
* GLUE The Multi-Genre NLI Corpus (MNLI) - `data/download/MNLI`
* BERT Large - `data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16`
* BERT Base - `data/download/google_pretrained_weights/uncased_L-12_H-768_A-12`
* BERT - `data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16`
* Wikipedia + BookCorpus TFRecords - `data/tfrecords<config>/books_wiki_en_corpus`

### Step 3. BERT Training 하기

BERT Training은 Pre-training과 Find-Tuinng 단계로 나뉘어 지게 됩니다. 

#### Pre-Training 시작하기 (스크립트)

BERT는 언어 표현을 위해 bidirectional representations을 pre-train하게 디자인되었습니다. 다음 스크립트는 Wikipedia와 BookCorpus 데이터셋으로 Pre-training을 진행합니다. 
두 개의 optimizer (LAMB / Adam)를 이용한 pre-training 스크립트를 제공하고 파라미터는 다음과 같습니다. 
* Pre-Training with LAMB paper 

`
scripts/run_pretraining_lamb.sh <train_batch_size_phase1> <train_batch_size_phase2> <eval_batch_size> <learning_rate_phase1> <learning_rate_phase2> <precision> <use_xla> <num_gpus> <warmup_steps_phase1> <warmup_steps_phase2> <train_steps> <save_checkpoint_steps> <num_accumulation_phase1> <num_accumulation_steps_phase2> <bert_model>
`

`run_pretraining_lamb.sh` 스크립트는 LAMB optimizer를 사용하고 두 개의 phase로 나눠서 pre-training을 진행합니다. 첫 phase는 sequence length를 128로 설정 한 뒤 전체 training step의 90%를 수행합니다. 두 번째 phase는 sequence length를 512로 설정 한 뒤 나머지 10%를 수행합니다. 

* Pre-Training with original BERT paper

`
scripts/run_pretraining_adam.sh <train_batch_size_per_gpu> <eval_batch_size> <learning_rate_per_gpu> <precision> <use_xla> <num_gpus> <warmup_steps> <train_steps> <save_checkpoint_steps> <num_accumulation_steps> <seq_len> <max_pred_per_seq>
`

`run_pretraining_adam.sh` 스크립트는 Adam optimizaer를 사용합니다. 
<a id='pre_train_param'></a>
파라미터 설명은 다음과 같습니다. 

* `<training_batch_size_phase*>` 각 respective phase에서의 GPU당 batch size, 큰 batch size는 Training의 효율성을 주지만, 더 많은 메모리를 요구함. 

* `<eval_batch_size>` Training 후 evaluation에서의 GPU 당 batch size 

* `<learning_rate_phase1>` batch size가 256일 때 Learning rate는 기본 1e-4 

* `<learning_rate_phase2>` batch size가 256일 때 Learning rate는 기본 1e-4 

* `<precision>` 모델의 precision(정밀도) fp32 또는 fp16을 선택

    * `fp32`: 32-bit IEEE single precision floats 사용
    * `fp16`: Automatic Mixed Precision 사용 
    
* `<use_xla>` XLA JIT 사용 여부
* `<num_gpus>` Traning 시 사용하는 GPU 개수. 노드의 GPU 개수보다 같거나 작아야 함.

* `<warmup_steps_phase*>` 처음 training을 시작하고 나서 warm-up step 개수

* `<training_steps>` 전체 Training step 

* `<save_checkpoint_steps>` checkpoints 저장 빈도( 기본 100, 100 step마다 checkpoint 저장)

* `<num_accumulation_phase*>` weight 업데이트 전에 gradient를 N번 누적하여 respective phase에서 더 높은 batch size를 가지는데 사용

* `<bert_model>` Pre-Training시 사용되는 BERT 모델 (large: BERT Large model, base: BERT base model) 

    * `large` : BERT Large model
    * `base` : BERT Base model
    
    
주의: TFRecord 셋으로 구성한 seq_length와 max_prediction_per_seq를 사용하여 모델을 Training하거나, 사용할 seq_length 와 max_predictions_per_seq에 맞는 TFRecord 셋을 구성해줘야 합니다. 

XLA과 LAMB optimizer를 사용한 V100 32G 4GPU 에서의 BERT Base FP16 Pre-training 예제

In [ ]:
%%bash 

NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

docker rm -f bert_training

docker run --rm -t \
    --runtime=nvidia \
    --name=bert_training \
    --net=host \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -e NVIDIA_VISIBLE_DEVICES=$NV_VISIBLE_DEVICES \
    -v $PWD:/workspace/bert \
    -v $PWD/results:/results \
    bert bash scripts/run_pretraining_lamb.sh 16 4 8 7.5e-4 5e-4 fp16 true 4 2000 200 7820 100 128 512 base
    

#### Pre-Training 시작하기 (커스텀)

BERT_CONFIG의 json 파일을 수정해서 BERT모델을 수정하실 수 있습니다. 
파라미터 설명은 [여기](#pre_train_param)을 참조 하시길 바랍니다. 


In [ ]:
%%bash 


NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

train_batch_size=${1:-14}
eval_batch_size=${2:-8}
learning_rate=${3:-"1e-4"}
precision=${4:-"manual_fp16"}
use_xla=${5:-"true"}
num_gpus=${6:-8}
warmup_steps=${7:-"10000"}
train_steps=${8:-1144000}
save_checkpoints_steps=${9:-5000}
bert_model=${10:-"base"}
num_accumulation_steps=${11:-1}
seq_len=${12:-128}
max_pred_per_seq=${13:-20}

DATA_DIR=data/tfrecord/lower_case_1_seq_len_${seq_len}_max_pred_${max_pred_per_seq}_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5_shard_1472_test_split_10/books_wiki_en_corpus

if [ "$bert_model" = "large" ] ; then
    export BERT_CONFIG=data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16/bert_config.json
else
    export BERT_CONFIG=data/download/google_pretrained_weights/uncased_L-12_H-768_A-12/bert_config.json
fi

PREC=""
if [ "$precision" = "fp16" ] ; then
   PREC="--use_fp16"
elif [ "$precision" = "fp32" ] ; then
   PREC=""
elif [ "$precision" = "manual_fp16" ] ; then
   PREC="--manual_fp16"
else
   echo "Unknown <precision> argument"
   exit -2
fi

if [ "$use_xla" = "true" ] ; then
    PREC="$PREC --use_xla"
    echo "XLA activated"
fi

export GBS=$(expr $train_batch_size \* $num_gpus \* $num_accumulation_steps)
printf -v TAG "tf_bert_pretraining_adam_%s_%s_gbs%d" "$bert_model" "$precision" $GBS
DATESTAMP=`date +'%y%m%d%H%M%S'`

#Edit to save logs & checkpoints in a different directory
RESULTS_DIR=${RESULTS_DIR:-/results/${TAG}_${DATESTAMP}}
LOGFILE=$RESULTS_DIR/$TAG.$DATESTAMP.log
mkdir -m 777 -p $RESULTS_DIR
printf "Saving checkpoints to %s\n" "$RESULTS_DIR"
printf "Logs written to %s\n" "$LOGFILE"

INPUT_FILES="$DATA_DIR/training"
EVAL_FILES="$DATA_DIR/test"

mpi=""
if [ $num_gpus -gt 1 ] ; then
   mpi="mpiexec --allow-run-as-root -np $num_gpus --bind-to socket"
fi

docker rm -f bert_training

docker run --rm -t \
    --runtime=nvidia \
    --name=bert_training \
    --net=host \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -e NVIDIA_VISIBLE_DEVICES=$NV_VISIBLE_DEVICES \
    -v $PWD:/workspace/bert \
    -v $PWD/results:/results \
    bert $mpi python3 /workspace/bert/run_pretraining.py \
     --input_files_dir=$INPUT_FILES \
     --eval_files_dir=$EVAL_FILES \
     --output_dir=$RESULTS_DIR \
     --bert_config_file=$BERT_CONFIG \
     --do_train=True \
     --do_eval=True \
     --train_batch_size=$train_batch_size \
     --eval_batch_size=$eval_batch_size \
     --max_seq_length=$seq_len \
     --max_predictions_per_seq=$max_pred_per_seq \
     --num_train_steps=$train_steps \
     --num_accumulation_steps=$num_accumulation_steps \
     --num_warmup_steps=$warmup_steps \
     --save_checkpoints_steps=$save_checkpoints_steps \
     --learning_rate=$learning_rate \
     --horovod $PREC \
     --allreduce_post_accumulation=True
     

#### Fine-tuning 시작하기 (스크립트)

FIne-Tuning 단계에서는 기존의 pre-trained BERT 모델에 하나의 output layer를 추가함으로써 Q&A 시스템 문제 및 GRUE 벤치마크를 수행할 수 있습니다. GLUE benchmakr와 Q&A 문제를 Fine-tuing 하기 위한 스크립트를 제공합니다. Pre-Training 단계에서 생성된 checkpoint를 파라미터로 넣어 학습을 진행해야 합니다. 

* `scripts/run_squad.sh`

Q&A 문제에 대한 fine-tuning 스트립트로써, SQuAD 데이터 셋을 사용하여 학습을 진행합니다. 사용가능한 SQuAD 데이터셋 버전은 1.1과 2.0입니다. 

`bash scripts/run_squad.sh <batch_size_per_gpu> <learning_rate_per_gpu> <precision> <use_xla> <num_gpus> <seq_length> <doc_stride> <bert_model> <squad_version> <checkpoint> <epochs>`

* 
`bash scripts/run_glue.sh`

GRUE 벤치마크 중 CoLA, MRPC,  MNLI를 지원합니다. 

`bash scripts/run_glue.sh <task_name> <batch_size_per_gpu> <learning_rate_per_gpu> <precision> <use_xla> <num_gpus> <seq_length> <doc_stride> <bert_model> <epochs> <warmup_proportion> <checkpoint>`
 
 <a id='fine_tuning_param'></a>
 Fine-tuning을 위한 파라미터는 다음과 같습니다.
 * `<batch_size_per_gpu>` GPU당 batch size
 * `<learning_rate_per_gpu>` Learning rate
 * `<precision>` 모델의 precision(정밀도) fp32 또는 fp16을 선택
 * `<use_xla>` XLA JIT 사용 여부
 * `<num_gpus> ` Traning 시 사용하는 GPU 개수
 * `<seq_length>` 최대 input sequence 길이 
 * `<doc_stride>` 긴 문서를 chunk로 나눌 때의 stride
 * `<bert_model>`   Pre-Training시 사용되는 BERT 모델 (large: BERT Large model, base: BERT base model) 
 * `<squad_version>`  SQuAD 데이터셋 버전 
     * 1.1: SQuAD v1.1
     * 2.0: SQuAD v2.0
 * `<checkpoint>`  pre-trained checkpoint
 * `<warmup_proportion>` linear learning rate warpup 실행을 위한 Training 비율 0.1 일 경우 10%

※__앞서 `<checkpoint>`에 Pre-Training으로 생성된 checkpoint를 사용하시길 바랍니다. 다음 실행 코드에서는 다운받은 pre-trained 모델 chekcpoint를 사용하였습니다. (pre-training 으로 생성된 checkpoint는 BERT 기본 폴더 내 results 폴더나 docker container 내에 /results 폴더에 위치하게 됩니다.)__

SQuAD v1.1 데이터셋을 이용한 V100 32G 4GPU 에서의 BERT FP16 Fine-tuning 예제 

In [ ]:
%%bash

NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

docker rm -f bert_training

docker run --rm -t \
    --runtime=nvidia \
    --name=bert_training \
    --net=host \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -e NVIDIA_VISIBLE_DEVICES=$NV_VISIBLE_DEVICES \
    -v $PWD:/workspace/bert \
    -v $PWD/results:/results \
    bert bash scripts/run_squad.sh 10 5e-6 fp16 true 4 384 128 base 1.1 data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16/bert_model.ckpt
    

#### Fine-tuning 시작하기 (커스텀)


SQuAD 데이터셋을 위한 코드로써 `run_squad.py` 파일을 실행해서 학습 할 수 있습니다. pre-trainig 단계에서 training된 checkpoint를 `init_checkpoint`에 설정해 주시길 바랍니다. 파라미터 설명은 [여기](#fine_tuning_param)을 참조 하시길 바랍니다. 


In [ ]:
%%bash


NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

batch_size=${1:-"8"}
learning_rate=${2:-"5e-6"}
precision=${3:-"fp16"}
use_xla=${4:-"true"}
num_gpu=${5:-"4"}
seq_length=${6:-"384"}
doc_stride=${7:-"128"}
bert_model=${8:-"large"}

if [ "$bert_model" = "large" ] ; then
    export BERT_DIR=data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16
else
    export BERT_DIR=data/download/google_pretrained_weights/uncased_L-12_H-768_A-12
fi

squad_version=${9:-"1.1"}

export SQUAD_DIR=data/download/squad/v${squad_version}
if [ "$squad_version" = "1.1" ] ; then
    version_2_with_negative="False"
else
    version_2_with_negative="True"
fi

init_checkpoint=${10:-"$BERT_DIR/bert_model.ckpt"}
epochs=${11:-"2.0"}

echo "Squad directory set as " $SQUAD_DIR " BERT directory set as " $BERT_DIR

use_fp16=""
if [ "$precision" = "fp16" ] ; then
        echo "fp16 activated!"
        use_fp16="--use_fp16"
fi

if [ "$use_xla" = "true" ] ; then
    use_xla_tag="--use_xla"
    echo "XLA activated"
else
    use_xla_tag=""
fi

if [ $num_gpu -gt 1 ] ; then
    mpi_command="mpirun -np $num_gpu -H localhost:$num_gpu \
    --allow-run-as-root -bind-to none -map-by slot \
    -x NCCL_DEBUG=INFO \
    -x LD_LIBRARY_PATH \
    -x PATH -mca pml ob1 -mca btl ^openib"
else
    mpi_command=""
fi

export GBS=$(expr $batch_size \* $num_gpu)
printf -v TAG "tf_bert_finetuning_squad_%s_%s_gbs%d" "$bert_model" "$precision" $GBS
DATESTAMP=`date +'%y%m%d%H%M%S'`

#Edit to save logs & checkpoints in a different directory
RESULTS_DIR=results/${TAG}_${DATESTAMP}
LOGFILE=$RESULTS_DIR/$TAG.$DATESTAMP.log
mkdir -m 777 -p $RESULTS_DIR
printf "Saving checkpoints to %s\n" "$RESULTS_DIR"
printf "Logs written to %s\n" "$LOGFILE"

#Check if all necessary files are available before training
for DIR_or_file in $SQUAD_DIR $RESULTS_DIR $BERT_DIR/bert_config.json $BERT_DIR/vocab.txt; do
  if [ ! -d "$DIR_or_file" ] && [ ! -f "$DIR_or_file" ]; then
     echo "Error! $DIR_or_file directory missing. Please mount correctly"
     exit -1
  fi
done

docker rm -f bert_training

docker run --rm -t \
    --runtime=nvidia \
    --name=bert_training \
    --net=host \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -e NVIDIA_VISIBLE_DEVICES=$NV_VISIBLE_DEVICES \
    -v $PWD:/workspace/bert \
    -v $PWD/results:/results \
    bert $mpi_command python run_squad.py \
    --vocab_file=$BERT_DIR/vocab.txt \
    --bert_config_file=$BERT_DIR/bert_config.json \
    --init_checkpoint=$init_checkpoint \
    --do_train=True \
    --train_file=$SQUAD_DIR/train-v${squad_version}.json \
    --do_predict=True \
    --predict_file=$SQUAD_DIR/dev-v${squad_version}.json \
    --train_batch_size=$batch_size \
    --learning_rate=$learning_rate \
    --num_train_epochs=$epochs \
    --max_seq_length=$seq_length \
    --doc_stride=$doc_stride \
    --save_checkpoints_steps 1000 \
    --output_dir=$RESULTS_DIR \
    --horovod "$use_fp16" \
    $use_xla_tag --version_2_with_negative=${version_2_with_negative} |& tee $LOGFILE; 
    python $SQUAD_DIR/evaluate-v${squad_version}.py $SQUAD_DIR/dev-v${squad_version}.json ${RESULTS_DIR}/predictions.json |& tee -a $LOGFILE
    
    

다음은 GLUe benchmark를 위한 코드로써 `run_classifier.py` 파일을 실행해서 학습 할 수 있습니다. `task_name`에 GLUE benchmark의 task (CoLA, MRPC,  MNLI)를 입력해주시면 됩니다. 또한 pre-trainig 단계에서 training된 checkpoint를 `init_checkpoint`에 설정해 주시길 바랍니다. 

파라미터 설명은 [여기](#fine_tuning_param)을 참조 하시길 바랍니다. 

In [ ]:
%%bash

NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

task_name=${1:-"MRPC"}
batch_size=${2:-"32"}
learning_rate=${3:-"2e-5"}
precision=${4:-"fp16"}
use_xla=${5:-"true"}
num_gpu=${6:-"8"}
seq_length=${7:-"128"}
doc_stride=${8:-"64"}
bert_model=${9:-"large"}

if [ "$bert_model" = "large" ] ; then
    export BERT_DIR=data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16
else
    export BERT_DIR=data/download/google_pretrained_weights/uncased_L-12_H-768_A-12
fi
export GLUE_DIR=data/download


epochs=${10:-"3.0"}
ws=${11:-"0.1"}
init_checkpoint=${12:-"$BERT_DIR/bert_model.ckpt"}

echo "GLUE directory set as " $GLUE_DIR " BERT directory set as " $BERT_DIR

use_fp16=""
if [ "$precision" = "fp16" ] ; then
        echo "fp16 activated!"
        use_fp16="--use_fp16"
fi

if [ "$use_xla" = "true" ] ; then
    use_xla_tag="--use_xla"
    echo "XLA activated"
else
    use_xla_tag=""
fi

if [ $num_gpu -gt 1 ] ; then
    mpi_command="mpirun -np $num_gpu -H localhost:$num_gpu \
    --allow-run-as-root -bind-to none -map-by slot \
    -x NCCL_DEBUG=INFO \
    -x LD_LIBRARY_PATH \
    -x PATH -mca pml ob1 -mca btl ^openib"
else
    mpi_command=""
fi

export GBS=$(expr $batch_size \* $num_gpu)
printf -v TAG "tf_bert_finetuning_glue_%s_%s_%s_gbs%d" "$task_name" "$bert_model" "$precision" $GBS
DATESTAMP=`date +'%y%m%d%H%M%S'`
#Edit to save logs & checkpoints in a different directory
RESULTS_DIR=results/${TAG}_${DATESTAMP}
LOGFILE=$RESULTS_DIR/$TAG.$DATESTAMP.log
mkdir -m 777 -p $RESULTS_DIR
printf "Saving checkpoints to %s\n" "$RESULTS_DIR"
printf "Logs written to %s\n" "$LOGFILE"

#Check if all necessary files are available before training
for DIR_or_file in $GLUE_DIR/${task_name} $RESULTS_DIR $BERT_DIR/vocab.txt $BERT_DIR/bert_config.json; do
  echo $DIR_or_file
  if [ ! -d "$DIR_or_file" ] && [ ! -f "$DIR_or_file" ]; then
     echo "Error! $DIR_or_file directory missing. Please mount correctly"
     exit -1
  fi
done

docker rm -f bert_training

docker run --rm -t \
    --runtime=nvidia \
    --name=bert_training \
    --net=host \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -e NVIDIA_VISIBLE_DEVICES=$NV_VISIBLE_DEVICES \
    -v $PWD:/workspace/bert \
    -v $PWD/results:/results \
    bert $mpi_command python3 run_classifier.py \
    --task_name=$task_name \
    --do_train=true \
    --do_eval=true \
    --data_dir=$GLUE_DIR/$task_name \
    --vocab_file=$BERT_DIR/vocab.txt \
    --bert_config_file=$BERT_DIR/bert_config.json \
    --init_checkpoint=$init_checkpoint \
    --max_seq_length=$seq_length \
    --doc_stride=$doc_stride \
    --train_batch_size=$batch_size \
    --learning_rate=$learning_rate \
    --num_train_epochs=$epochs \
    --output_dir=$RESULTS_DIR \
    --horovod "$use_fp16" \
    $use_xla_tag --warmup_proportion=$ws |& tee $LOGFILE
    


**TensorRT로 변환하기 위한 파일 목록입니다. 꼭 TensorRT 적용하기 전에 확인해주세요**
* `checkpoint`: 모든 Pre-training 과 fune-tuning 의 checkpoint는 `./results` 폴더에 저장됩니다. 
* `BERT_config.json`: BERT 모델의 layer정보로써 이 예제에서는 BERT base 인 경우 `./data/download/google_pretrained_weights/uncased_L-12_H-768_A-12`, BERT large 인 경우 `./data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16` 를 사용했습니다. 
* `vocab.txt` : vocabulary 정보보써 이 예제에서는 BERT base 인 경우 `data/download/google_pretrained_weights/uncased_L-12_H-768_A-12`, BERT large 인 경우 `./data/download/google_pretrained_weights/uncased_L-24_H-1024_A-16` 를 참고해주세요 

<a id='performance'></a>
## 5. Training 성능

<img src="../data/images/time_single_node.PNG" width="500">
그림 3. single-node에서 DGX-1와 DGX-2의 training 시간 비교

그림 3은 DGX-1과 DGX-2에서의 Pre-training 단계에서 Mixed precision training 시간을 비교한 것으로, DGX-2에서 약 2배의 training 시간 향상을 가져옵니다.
<img src="../data/images/time_multi_node.PNG" width="500">
그림 4. Multi-node에서 DGX-1와 DGX-2H의 training 시간 비교

그림 4는 Multi-node에서 DGX-1과 DGX-2H의 Mixed Precision Trainig 시간을 비교한 것으로 4, 16, 32개의 node에서 DGX-2H가 DGX-1보다 각각   2.8, 2.6, 2.9배의 trianing 시간이 향상되었습니다.

<img src="../data/images/time_multi_node_precision.PNG" width="500">
그림 5. 64개 node에서 Mixed Precision과 FP32 간의 training 시간 비교 

그림 5는 DGX-2H 64개의 노드에서 Single Precision과 Mixed Precision Trainig 시간을 비교한 Mixed Precision Trianig 시간이 Single Precisin Trainig 시간에 비해 약 2.3배 향상되었습니다. BERT의 Pre-training 단계의 trainig 시간을 3시간 이내에 마칠수 있음을 의미합니다. 

자세한 성능과 성능 측정 파라미터 값은 [여기](https://github.com/NVIDIA/DeepLearningExamples/tree/master/TensorFlow/LanguageModeling/BERT#performance)를 참고 하시길 바랍니다. 



<a id='conclusions'></a>
## 6. 결론

지금까지  BERT 모델 아키텍처에 설명, BERT Training에 필요한 단계인 Pre-training과 Fine-tuning 각 단계에서의 Training 과정에 대해 설명했습니다. 이제 여러분께서는 BERT Triaing 과정을 모두 마치셨습니다. BERT End-to-End의 다음 과정은 Low latency & High throughput을 위한 Infernece와 Inference service를 위한 TensorRT 와 TensorRT inference server 적용 과정을 설명 드리겠습니다. 